Interactive control examples.

-Brij Bhushan  
16-Sep-2019

# Initialize

In [1]:
#Load packages
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import MaxNLocator
import control as ctl

from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

In [2]:
#Define params
s = ctl.tf([1,0],[1]) #Define Laplace 's' variable
π = np.pi;

# 1st-Order transfer function

In [3]:
def get1stOrderPlant(tauz=10,taup=1,K=1):
    '''
    Plots the Bode-plot, pz-map and step response of first order bode plots.
    1/tauz: zero location [rad/s],
    1/taup: pole location [rad/s],
    K : DC-Gain [mag]
    '''
    plant = K*(tauz*s+1)/(taup*s+1)
   
    return plant

def plot1stOrderPlots(tau_z=10,tau_p=1,K=1):
    '''
    Encapsulate get1stOrderPlant to add flexibility.
    Plots the various plots
    '''
    plant = get1stOrderPlant(tau_z,tau_p,K);
    print("TF:\n")
    print(plant)
    t, yout = ctl.step_response(plant)
    fig=plt.figure(1)
    plt.plot(t,yout)
    plt.plot([0,0,t[-1]],[0,1,1],alpha=0.5)
    plt.title('Step Response');
    plt.ylabel('Response');
    plt.xlabel('Time [seconds]');
    plt.show()
    plt.ylim(0); #adjust limits to not show the negative values
#     plt.xlim([0,time[(x_0*1e6+response)<=0][0]]); #adjust limits to not show the negative values
    ctl.bode_plot(plant,omega_num=10000,Hz=False); plt.suptitle('Bode Plot');
    ctl.pzmap(plant)
    return

In [4]:
interact(plot1stOrderPlots, tau_z=(0,100,1),tau_p=(1,100,1), K=(1,100));

interactive(children=(IntSlider(value=10, description='tau_z'), IntSlider(value=1, description='tau_p', min=1)…

# 2nd-Order Plant

In [5]:
def get2ndOrderPlant(ω0,ζ):
    '''
    ω0: zero location [rad/s],
    ζ: pole location [rad/s],
    '''
    plant = 1/((s/ω0)**2+2*ζ/ω0*s+1);
    return plant

def plot2ndOrderPlots(ω0=1,ζ=0.5):
    '''
    Encapsulate get2ndOrderPlant to add flexibility.
    Plots the Bode-plot, pz-map and step response of 2nd-order bode plots.
    '''
    plant = get2ndOrderPlant(ω0,ζ);
    print("TF: ")
    print(plant)
#     print('poles:')
#     print(ctl.damp(plant));
    t, yout = ctl.step_response(plant);
    t = np.linspace(0,t[-1],1000);
    t, yout = ctl.step_response(plant,t)
    fig=plt.figure(1)
    plt.plot(t,yout)
    plt.plot([0,0,t[-1]],[0,1,1],alpha=0.5)
    plt.title('Step Response');
    plt.ylabel('Response');
    plt.xlabel('Time [seconds]');
    plt.show()
    plt.ylim(0); #adjust limits to not show the negative values
#     plt.xlim([0,time[(x_0*1e6+response)<=0][0]]); #adjust limits to not show the negative values
    ctl.bode_plot(plant,omega_num=10000,Hz=False); plt.suptitle('Bode Plot');
    ctl.pzmap(plant)
    return

In [6]:
interact(plot2ndOrderPlots,ω0=(1,100,1),ζ=(0,1,0.01));

interactive(children=(IntSlider(value=1, description='ω0', min=1), FloatSlider(value=0.5, description='ζ', max…

# Discrete time integrator with Kp

Plant: $H(z) = \frac{1}{z-1}$, $T=1$  
Ctrl: $K_p$ (pure proportional controller)  
RR: $\frac{K_p}{z-1}$, $T=1$


In [7]:
def getDiscIntPlant(Ts=1):
    '''
    K: slope of integrator
    '''
    z = ctl.tf([1,0],[1],Ts)
    plant = 1/(z-1);
    return plant

def getDiscIntCLplots(Kp=0.5):
    '''
    Encapsulate getDiscIntPlant to add flexibility, close the loop and 
    plot various relevant plots.
    '''
    plant = getDiscIntPlant(Ts=1);
#     print("plant TF: ")
#     print(plant)

    RR = Kp*plant;
#     print("RR TF: ")
#     print(RR)
    
    CL = ctl.feedback(RR,1)
    
    #Step-repose of closed loop
    t=np.arange(0,21,1)
    t, yout = ctl.step_response(CL,t);
    fig=plt.figure(1); ax=fig.gca();
    plt.step(t,np.squeeze(yout),where='post')
    plt.plot([0,0,t[-1]],[0,1,1],alpha=0.5)
    plt.ylim(0); #adjust limits to not show the negative values
    plt.xticks(np.arange(0,21,2));
    plt.title('CL Step Response');
    plt.ylabel('Response');
    plt.xlabel('Sample #');
    plt.show()
#     ax.xaxis.set_major_locator(MaxNLocator(integer=True,nbins=21))
#     plt.xlim(0,20)
    
    #Bode plot of return-ratio
    ctl.bode_plot(RR,omega_num=10000,Hz=False,margins=False); plt.suptitle('RR Bode Plot');
    
    #PZ-map of the closed loop
    ctl.pzmap(CL, title = 'CL PZ-Map')
    #Draw unit circle for the z-plane
    circle1 = plt.Circle((0, 0), 1, alpha=0.05)
    fig = plt.gcf(); ax = fig.gca(); ax.add_artist(circle1);
    ax.set_aspect('equal', 'box');
    plt.xlim(-1.1,1.1)
    plt.ylim(-1.1,1.1)
    return

In [8]:
interact(getDiscIntCLplots,Kp=(0.01,2,0.01));

interactive(children=(FloatSlider(value=0.5, description='Kp', max=2.0, min=0.01, step=0.01), Output()), _dom_…